In [1]:
# 문장 감정 분류 CNN 모델 사용 
import tensorflow as tf 
import pandas as pd 
from tensorflow.keras.models import Model, load_model 
from tensorflow.keras import preprocessing 

# 데이터 읽어오기 
# pandas read_csv() 함수 이용. ChatbotData.csv 파일 읽어와 label(감정) 분류할 Q(질문) 데이터 features 리스트에 저장. 
# label 리스트는 CNN 모델이 예측한 분류 결과와 실제 분류값을 비교하기 위한 목적 사용
train_file = './data/ChatbotData.csv' 
data = pd.read_csv(train_file, delimiter=',') 
features = data['Q'].tolist() 
labels = data['label'].tolist() 

# 단어 인덱스 시퀀스 벡터 
# 질문리스트 features 에서 한 문장씩 꺼내와 text_to_word_sequence() 함수 이용해 단어 시퀀스 만든 후 말뭉치(corpus) 리스트 저장 
# tensorflow 토크나이저 texts_to_sequences() 함수 이용해 문장 내 모든 단어 시퀀스 번호로 변환 
# 단어 시퀀스 벡터 크기 맞추기 위해 pad_sequences() 함수 사용하여 패딩 처리 
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features] 
tokenizer = preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(corpus) 
sequences = tokenizer.texts_to_sequences(corpus) 

MAX_SEQ_LEN = 15   # 단어 시퀀스 벡터 크기 
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding = 'post') 

# 테스트용 데이터셋 생성 
# 위에서 처리한 시퀀스(padded_seqs) 벡터 리스트와 감정(labels) 리스트 전체를 데이터셋 객체로 만듦 
# 데이터 랜덤으로 섞은 후 테스트용 데이터셋 2,000개 뽑아서 20개씩 처리 
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels)) 
ds = ds.shuffle(len(features)) 
test_ds = ds.take(2000).batch(20)   # 테스트 데이터셋 

# 감정 분류 CNN 모델 불러오기
# 케라스 load_model() 함수 사용 모델 파일 불러오기 
# 성공적 불러오면 학습 모델 객체 반환.
# 파일 저장된 모델 정보 확인 summary() 함수 호출. 테스트셋 데이터 이용 모델 성틍 평가
model = load_model('cnn_model.h5') 
model.summary() 
model.evaluate(test_ds, verbose = 2) 

#테스트용 데이터셋의 10212 번째 데이터 출력 
# 망뭉치 데이터 리스트 10212번째 문장 감정 예측
# 예측 앞서 데이터 확인
print('단어 시퀀스 :', corpus[10212]) 
print('단어 인덱스 시퀀스 :', padded_seqs[10212]) 
print('문장 분류(정답) :', labels[10212]) 

# 테스트용 데이터셋의 10212번째 데이터 검증 예측 
# 케라스 predict() 함수는 입력 데이터에 대해 각 클래스별로 예측한 점수를 반환
# 텐서플로 argmax() 함수 이용해 분류 클래스들 중 예측 점수가 가장 큰 클래스 번호 계산 = 10212번째 문장이 어떤 감정 클래스에 포함되어 있는지 판단 
picks = [10212] 
predict = model.predict(padded_seqs[picks]) 
predict_class = tf.math.argmax(predict, axis = 1) 
print('감정 예측 점수 :', predict) 
print('감정 예측 클래스 :', predict_class.numpy()) 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 15, 128)      1715072     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 15, 128)      0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 13, 128)      49280       ['dropout[0][0]']                
                                                                                              